# Fake News Generator & Detector using GPT-2 and BERT

This project showcases a dual AI system — one that generates fake news headlines using GPT-2, and another that detects them using a fine-tuned BERT classifier. It creatively demonstrates both the capabilities and ethical challenges of generative AI, especially in combating misinformation.
Built using Hugging Face Transformers, Gradio, and hosted on Google Colab for instant demo access.

In [1]:
!pip install transformers gradio datasets torch scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer, BertTokenizer, BertForSequenceClassification
from transformers import TextClassificationPipeline
import torch
import gradio as gr
import random
import numpy as np


In [3]:
generator = pipeline("text-generation", model="gpt2", tokenizer="gpt2")

def generate_fake_news(prompt):
    response = generator(prompt, max_length=30, num_return_sequences=1, do_sample=True, temperature=0.9)
    return response[0]['generated_text']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [4]:
detector = pipeline("text-classification", model="mrm8488/bert-tiny-finetuned-fake-news-detection", tokenizer="bert-base-uncased")

def detect_news_label(text):
    result = detector(text)[0]
    label = result['label']
    score = round(result['score'] * 100, 2)
    label_text = "🟢 REAL" if label == "REAL" else "🔴 FAKE"
    return f"{label_text} ({score}%)"


config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


In [5]:
def full_pipeline(prompt):
    fake_headline = generate_fake_news(prompt)
    label = detect_news_label(fake_headline)
    return fake_headline, label

demo = gr.Interface(
    fn=full_pipeline,
    inputs=gr.Textbox(placeholder="Enter topic (e.g., Politics, Technology, etc.)", label="News Topic Prompt"),
    outputs=[
        gr.Textbox(label="📰 Generated Headline"),
        gr.Textbox(label="🔎 Detected Label")
    ],
    title="📰 Fake News Generator & Detector",
    description="⚡ Generate a fake news headline using GPT-2 and detect whether it's fake or real using a BERT classifier.\nThis demo highlights the dual power & responsibility of Generative AI in tackling misinformation.",
    theme="soft",
    examples=[["Climate Change"], ["Election 2024"], ["Aliens landed in California!"]]
)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1263cf404afa47e54a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
